In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

import pandas as pd
import numpy as np

import datetime
from dateutil.relativedelta import relativedelta 

import random

from catboost import CatBoostClassifier

from tqdm.auto import tqdm

from utils import add_time_series_features, add_master_data_mappings, evaluate

tqdm.pandas()

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

In [2]:
data_path = "./data/fact_train_test.csv"
data = pd.read_csv(data_path, sep=";", decimal=",", encoding="windows-1251")
data

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,real_wagon_count
0,2012-07-01,1,5,1,328,1193,30252,13005,10036,71.0,1
1,2012-10-01,1,5,1,328,1193,30252,11376,10036,210.0,3
2,2014-03-01,0,1,1,328,3472,30252,29548,10036,67.0,1
3,2014-03-01,0,1,1,328,3472,30252,29158,10036,67.0,1
4,2014-03-01,0,2,1,328,3472,30252,27484,10036,66.0,1
...,...,...,...,...,...,...,...,...,...,...,...
3559227,2023-03-01,1,5,2,1346,1482,31438,31482,0,69.0,1
3559228,2023-03-01,1,5,2,1346,1492,31438,36091,27275,70.0,1
3559229,2023-03-01,1,5,2,1346,1492,31438,35450,27275,70.0,1
3559230,2023-03-01,1,5,2,1346,1492,31438,31482,0,207.0,3


### Preprocessing

In [3]:
data["period"] = pd.to_datetime(data["period"], format="%Y-%m-%d", errors='coerce')

In [4]:
date_min = data["period"].min()
date_max = data["period"].max()
data["months_from_start"] = (data["period"].dt.year - date_min.year) * 12 + (data["period"].dt.month - date_min.month)
data

data = data.drop(columns=["period"])

In [5]:
accuracy_granularity = [
    # "period",
    "rps",
    # "holding_name",
    # "sender_department_name",
    # "recipient_department_name",
    
    "client_sap_id",
    "freight_id",
    "sender_station_id",
    "recipient_station_id",
    "sender_organisation_id",
    "podrod",
    "filial",
]

In [6]:
data_grouped = data.groupby(accuracy_granularity).agg(list)
data_grouped

real_weight  \
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                   
0   -1            0          2977              38567                7152                   0      2               [31.0]   
                             3801              26930                7152                   1      2               [90.0]   
                             3845              28207                8535                   1      2                [4.0]   
                             4602              31057                0                      0      2               [40.0]   
                             4871              25250                0                      1      2         [68.0, 68.0]   
...                                                                                                                  ...   
1    2415         2997       13005             33625                10441                  5      2       [136.0, 138.0]   
                                               37870                10441                  5      2        [70.0, 137.0]   
                  3471       32800             12142                7178                   5      2               [67.0]   
                                               12840                7178                   5      2               [67.0]   
     2416         1691       27096             27484                10579                  5      2       [130.0, 130.0]   

                                                                                                         real_wagon_count  \
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                    
0   -1            0          2977              38567                7152                   0      2                   [1]   
                             3801              26930                7152                   1      2                   [4]   
                             3845              28207                8535                   1      2                   [1]   
                             4602              31057                0                      0      2                   [2]   
                             4871              25250                0                      1      2                [1, 1]   
...                                                                                                                   ...   
1    2415         2997       13005             33625                10441                  5      2                [2, 2]   
                                               37870                10441                  5      2                [1, 2]   
                  3471       32800             12142                7178                   5      2                   [1]   
                                               12840                7178                   5      2                   [1]   
     2416         1691       27096             27484                10579                  5      2                [2, 2]   

                                                                                                         months_from_start  
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                    
0   -1            0          2977              38567                7152                   0      2                   [83]  
                             3801              26930                7152                   1      2                   [76]  
                             3845              28207                8535                   1      2                   [74]  
                             4602              31057                0                      0      2                   [55]  
                             4871              25250                0                      1      2               [62, 6

In [7]:
not_grouped_indexes = list(data_grouped.columns)
not_grouped_indexes

['real_weight', 'real_wagon_count', 'months_from_start']

In [8]:
for months in data_grouped["months_from_start"].tolist():
    if len(set(months)) != len(months):
        raise

In [9]:
# data_grouped = data_grouped.reset_index()
# data_grouped

In [10]:
k_ns = 3

def negative_sampling(column):
    if column.name == "real_weight":
        column = [line + [0] * k_ns for line in column]
    elif column.name == "real_wagon_count":
        column = [line + [0] * k_ns for line in column]
    elif column.name == "months_from_start":
        column = [line + random.sample(list(set(range(54, 128 + 1)) - set(line)), k_ns) for line in column]
    return column

In [11]:
data_with_ns = data_grouped.copy()
data_with_ns[not_grouped_indexes] = data_grouped[not_grouped_indexes].progress_apply(negative_sampling)

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
data_with_ns = data_with_ns.explode(not_grouped_indexes).reset_index()

In [13]:
data_with_ns["is_history"] = ((data_with_ns["real_weight"] != 0) | (data_with_ns["real_wagon_count"] != 0)).astype(int)

In [14]:
data_with_ns = add_master_data_mappings(data_with_ns)

In [15]:
data_with_ns

,rps,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,podrod,filial,real_weight,real_wagon_count,months_from_start,is_history,holding_name,freight_group_name,sender_department_name,sender_railway_name,recipient_department_name,recipient_railway_name
0,0,-1,0,2977,38567,7152,0,2,31.0,1,83,1,NaN,0,27,25,76,6
1,0,-1,0,2977,38567,7152,0,2,0,0,68,0,NaN,0,27,25,76,6
2,0,-1,0,2977,38567,7152,0,2,0,0,57,0,NaN,0,27,25,76,6
3,0,-1,0,2977,38567,7152,0,2,0,0,90,0,NaN,0,27,25,76,6
4,0,-1,0,3801,26930,7152,1,2,90.0,4,76,1,NaN,0,130,25,123,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6635721,1,2416,1691,27096,27484,10579,5,2,130.0,2,105,1,1935.0,39,86,39,94,39
6635722,1,2416,1691,27096,27484,10579,5,2,130.0,2,106,1,1935.0,39,86,39,94,39
6635723,1,2416,1691,27096,27484,10579,5,2,0,0,115,0,1935.0,39,86,39,94,39
6635724,1,2416,1691,27096,27484,10579,5,2,0,0,126,0,1935.0,39,86,39,94,39


In [16]:
data_with_ns["period"] = (date_min + data_with_ns["months_from_start"].progress_apply(lambda x: relativedelta(months=x)))
data_with_ns

  0%|          | 0/6635726 [00:00<?, ?it/s]

,rps,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,podrod,filial,real_weight,real_wagon_count,months_from_start,is_history,holding_name,freight_group_name,sender_department_name,sender_railway_name,recipient_department_name,recipient_railway_name,period
0,0,-1,0,2977,38567,7152,0,2,31.0,1,83,1,NaN,0,27,25,76,6,2019-06-01 00:00:00
1,0,-1,0,2977,38567,7152,0,2,0,0,68,0,NaN,0,27,25,76,6,2018-03-01 00:00:00
2,0,-1,0,2977,38567,7152,0,2,0,0,57,0,NaN,0,27,25,76,6,2017-04-01 00:00:00
3,0,-1,0,2977,38567,7152,0,2,0,0,90,0,NaN,0,27,25,76,6,2020-01-01 00:00:00
4,0,-1,0,3801,26930,7152,1,2,90.0,4,76,1,NaN,0,130,25,123,31,2018-11-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6635721,1,2416,1691,27096,27484,10579,5,2,130.0,2,105,1,1935.0,39,86,39,94,39,2021-04-01 00:00:00
6635722,1,2416,1691,27096,27484,10579,5,2,130.0,2,106,1,1935.0,39,86,39,94,39,2021-05-01 00:00:00
6635723,1,2416,1691,27096,27484,10579,5,2,0,0,115,0,1935.0,39,86,39,94,39,2022-02-01 00:00:00
6635724,1,2416,1691,27096,27484,10579,5,2,0,0,126,0,1935.0,39,86,39,94,39,2023-01-01 00:00:00


In [17]:
data_with_ns, date_features = add_time_series_features(data_with_ns)
data_with_ns = data_with_ns.drop(columns=["period_day", "period_seconds"])

In [18]:
data_with_ns = data_with_ns.fillna(-1)

In [19]:
data_with_ns["is_test"] = (data_with_ns["months_from_start"] >= 124).astype(int)
data_with_ns = data_with_ns[data_with_ns["months_from_start"] >= 54]
data_with_ns

,rps,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,podrod,filial,real_weight,real_wagon_count,...,freight_group_name,sender_department_name,sender_railway_name,recipient_department_name,recipient_railway_name,period,period_year,period_weekday,period_month,is_test
0,0,-1,0,2977,38567,7152,0,2,31.0,1,...,0,27,25,76,6,2019-06-01,2019,5,6,0
1,0,-1,0,2977,38567,7152,0,2,0.0,0,...,0,27,25,76,6,2018-03-01,2018,3,3,0
2,0,-1,0,2977,38567,7152,0,2,0.0,0,...,0,27,25,76,6,2017-04-01,2017,5,4,0
3,0,-1,0,2977,38567,7152,0,2,0.0,0,...,0,27,25,76,6,2020-01-01,2020,2,1,0
4,0,-1,0,3801,26930,7152,1,2,90.0,4,...,0,130,25,123,31,2018-11-01,2018,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6635721,1,2416,1691,27096,27484,10579,5,2,130.0,2,...,39,86,39,94,39,2021-04-01,2021,3,4,0
6635722,1,2416,1691,27096,27484,10579,5,2,130.0,2,...,39,86,39,94,39,2021-05-01,2021,5,5,0
6635723,1,2416,1691,27096,27484,10579,5,2,0.0,0,...,39,86,39,94,39,2022-02-01,2022,1,2,0
6635724,1,2416,1691,27096,27484,10579,5,2,0.0,0,...,39,86,39,94,39,2023-01-01,2023,6,1,1


### CatBoost

In [20]:
train = data_with_ns[data_with_ns["is_test"] == 0]
test = data_with_ns[data_with_ns["is_test"] == 1]

X_train = train.drop(columns=["real_weight", "real_wagon_count", "is_history", "is_test"])
y_train = train["is_history"].tolist()
X_test = test.drop(columns=["real_weight", "real_wagon_count", "is_history", "is_test"])
y_test = test["is_history"].tolist()

In [21]:
model = CatBoostClassifier(iterations=1000,
                           eval_metric="Accuracy",
                           use_best_model=True,
                           random_seed=42,
                           max_depth=2,
                           # learning_rate=0.1
)
# cat_features=["rps", "podrod", "filial", "client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id"] + cat_datetime_fe,
model.fit(X_train,
          y_train,
          eval_set=(X_test, y_test),
          verbose=100)

Learning rate set to 0.273988
0:	learn: 0.6762741	test: 0.8649905	best: 0.8649905 (0)	total: 256ms	remaining: 4m 16s
100:	learn: 0.7397450	test: 0.8902685	best: 0.8908464 (25)	total: 15.5s	remaining: 2m 17s
200:	learn: 0.7532764	test: 0.8905797	best: 0.8917589 (107)	total: 30.4s	remaining: 2m
300:	learn: 0.7568635	test: 0.8863051	best: 0.8917589 (107)	total: 45.5s	remaining: 1m 45s
400:	learn: 0.7602542	test: 0.8705824	best: 0.8917589 (107)	total: 1m	remaining: 1m 30s
500:	learn: 0.7621632	test: 0.8675057	best: 0.8917589 (107)	total: 1m 15s	remaining: 1m 15s
600:	learn: 0.7637418	test: 0.8642699	best: 0.8917589 (107)	total: 1m 31s	remaining: 1m
700:	learn: 0.7668487	test: 0.8623092	best: 0.8917589 (107)	total: 1m 45s	remaining: 45.1s
800:	learn: 0.7685046	test: 0.8649133	best: 0.8917589 (107)	total: 2m	remaining: 30s


KeyboardInterrupt: 

In [ ]:
# forecast = X_test.copy()
# forecast["forecast_wagon_count"] = model.predict(X_test)
# forecast = forecast[forecast["forecast_wagon_count"] > 0]

# fact = X_test.copy()
# fact["real_wagon_count"] = test["real_wagon_count"].tolist()

In [ ]:
# evaluate(fact, forecast, public=True), evaluate(fact, forecast, public=False)

In [23]:
model = CatBoostClassifier(iterations=120,
                           eval_metric="Accuracy",
                           # use_best_model=True,
                           random_seed=42,
                           max_depth=2,
                           learning_rate=0.273988
)

full_X = data_with_ns.drop(columns=["real_weight", "real_wagon_count", "is_history", "is_test"])
full_y = data_with_ns["is_history"].tolist()

model.fit(full_X,
          full_y,
          verbose=50
)

0:	learn: 0.5599704	total: 99.7ms	remaining: 11.9s
50:	learn: 0.7241550	total: 4.51s	remaining: 6.1s
100:	learn: 0.7407600	total: 8.89s	remaining: 1.67s
119:	learn: 0.7428216	total: 10.6s	remaining: 0us


### Predict

In [ ]:
data_grouped_predict = data_grouped[data_grouped["months_from_start"].apply(max) >= 54]
# TODO: Change 54 to 2020-01-01

In [25]:
date = datetime.datetime.strptime("2020-01-01", "%Y-%m-%d")
(data["period"].dt.year - date_min.year) * 12 + (data["period"].dt.month - date_min.month)

KeyError: 'period'

In [38]:
data_grouped_predict = data_grouped_predict.drop(columns=["real_weight", "real_wagon_count"])
data_grouped_predict["months_from_start"] = [[129, 130, 131, 132, 133] for _ in range(data_grouped_predict.shape[0])]
data_grouped_predict

months_from_start
rps client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id podrod filial                           
0   -1            0          2977              38567                7152                   0      2       [129, 130, 131, 132, 133]
                             3801              26930                7152                   1      2       [129, 130, 131, 132, 133]
                             3845              28207                8535                   1      2       [129, 130, 131, 132, 133]
                             4602              31057                0                      0      2       [129, 130, 131, 132, 133]
                             4871              25250                0                      1      2       [129, 130, 131, 132, 133]
...                                                                                                                             ...
1    2415         2997       13005             33625                10441                  5      2       [129, 130, 131, 132, 133]
                                               37870                10441                  5      2       [129, 130, 131, 132, 133]
                  3471       32800             12142                7178                   5      2       [129, 130, 131, 132, 133]
                                               12840                7178                   5      2       [129, 130, 131, 132, 133]
     2416         1691       27096             27484                10579                  5      2       [129, 130, 131, 132, 133]

[1024551 rows x 1 columns]

In [39]:
data_future_months = data_grouped_predict.explode(["months_from_start"]).reset_index()

In [40]:
data_future_months = add_master_data_mappings(data_future_months)

In [41]:
data_future_months["period"] = (date_min + data_future_months["months_from_start"].progress_apply(lambda x: relativedelta(months=x)))
data_future_months

  0%|          | 0/5122755 [00:00<?, ?it/s]

,rps,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,podrod,filial,months_from_start,holding_name,freight_group_name,sender_department_name,sender_railway_name,recipient_department_name,recipient_railway_name,period
0,0,-1,0,2977,38567,7152,0,2,129,NaN,0,27,25,76,6,2023-04-01 00:00:00
1,0,-1,0,2977,38567,7152,0,2,130,NaN,0,27,25,76,6,2023-05-01 00:00:00
2,0,-1,0,2977,38567,7152,0,2,131,NaN,0,27,25,76,6,2023-06-01 00:00:00
3,0,-1,0,2977,38567,7152,0,2,132,NaN,0,27,25,76,6,2023-07-01 00:00:00
4,0,-1,0,2977,38567,7152,0,2,133,NaN,0,27,25,76,6,2023-08-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5122750,1,2416,1691,27096,27484,10579,5,2,129,1935.0,39,86,39,94,39,2023-04-01 00:00:00
5122751,1,2416,1691,27096,27484,10579,5,2,130,1935.0,39,86,39,94,39,2023-05-01 00:00:00
5122752,1,2416,1691,27096,27484,10579,5,2,131,1935.0,39,86,39,94,39,2023-06-01 00:00:00
5122753,1,2416,1691,27096,27484,10579,5,2,132,1935.0,39,86,39,94,39,2023-07-01 00:00:00


In [42]:
data_future_months, date_features = add_time_series_features(data_future_months)
data_future_months = data_future_months.drop(columns=["period_day", "period_seconds"])

In [43]:
data_future_months = data_future_months.fillna(-1)

In [44]:
data_future_months["predicts"] = model.predict(data_future_months)

In [48]:
data_future_months = data_future_months[data_future_months["predicts"] == 1].reset_index()

In [67]:
# list(data_future_months[data_grouped.index.names].itertuples(index=False))

In [73]:
data_future_months["forecast_wagon_count"] = data_grouped.loc[list(data_future_months[data_grouped.index.names].itertuples(index=False))].reset_index()["real_wagon_count"].apply(lambda x: x[-1])
data_future_months["forecast_weight"] = data_grouped.loc[list(data_future_months[data_grouped.index.names].itertuples(index=False))].reset_index()["real_weight"].apply(lambda x: x[-1])
data_future_months

,index,rps,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,podrod,filial,months_from_start,...,sender_department_name,sender_railway_name,recipient_department_name,recipient_railway_name,period,period_year,period_weekday,period_month,predicts,forecast_wagon_count
0,1690,0,-1,0,25164,25304,27020,2,2,129,...,127,1,1,1,2023-04-01,2023,5,4,1,1
1,1691,0,-1,0,25164,25304,27020,2,2,130,...,127,1,1,1,2023-05-01,2023,0,5,1,1
2,1692,0,-1,0,25164,25304,27020,2,2,131,...,127,1,1,1,2023-06-01,2023,3,6,1,1
3,1693,0,-1,0,25164,25304,27020,2,2,132,...,127,1,1,1,2023-07-01,2023,5,7,1,1
4,1694,0,-1,0,25164,25304,27020,2,2,133,...,127,1,1,1,2023-08-01,2023,1,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191070,5119895,1,2413,1691,13770,14539,30142,5,2,129,...,10,30,171,30,2023-04-01,2023,5,4,1,1
1191071,5119896,1,2413,1691,13770,14539,30142,5,2,130,...,10,30,171,30,2023-05-01,2023,0,5,1,1
1191072,5119897,1,2413,1691,13770,14539,30142,5,2,131,...,10,30,171,30,2023-06-01,2023,3,6,1,1
1191073,5119898,1,2413,1691,13770,14539,30142,5,2,132,...,10,30,171,30,2023-07-01,2023,5,7,1,1


In [80]:
example = pd.read_csv("./data/forecast_example.csv", sep=";", decimal=",", encoding="windows-1251")

In [83]:
submission = data_future_months[example.columns]

In [88]:
submission.to_csv("./submission/classifier_0.csv", sep=";", decimal=",", encoding="windows-1251", index=False)